In [1]:
import requests
import os
from dotenv import load_dotenv
import base64
import numpy as np
import cv2
from uuid import uuid4
from pathlib import Path
from pymongo import MongoClient

In [2]:
load_dotenv()

True

In [3]:
URL = "http://localhost:8000/"
AUTHORIZATION_KEY = os.getenv("AUTHORIZATION_KEY")

In [4]:
mongo = MongoClient(os.getenv("MONGO"))

In [5]:
db = mongo.data.images

In [6]:
def make_api_request(endpoint, method, data=None, session_id=None):
    headers = {"Authorization": AUTHORIZATION_KEY}
    if session_id:
        headers["SessionId"] = session_id

    response = requests.request(method, URL + endpoint, headers=headers, json=data)
    return response

In [4]:
def process_image(image: str, show=True, save=False, name=None, path="uploaded_images"):
    header, base64_str = image.split(",", 1)
    image_bytes = base64.b64decode(base64_str)
    nparr = np.frombuffer(image_bytes, np.uint8)
    im = cv2.imdecode(nparr, cv2.IMREAD_COLOR)

    if im is None:
        return "Could not decode image"

    file_extension = header.split(";")[0].split("/")[1]
    filename = f"{name if name else uuid4()}.{file_extension}"
    file_path = Path(path) / filename

    if save:
        cv2.imwrite(str(file_path), im)

    if show:
        cv2.imshow("Image", im)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

In [24]:
session_id = make_api_request("start", "PUT").json()["SessionId"]
session_id

'0af39271-e1ce-42ca-be33-30e84ba5fe4e'

In [ ]:
image_data = db.find_one({"count": 1, "name": "Niksh"})["image"]

In [5]:
image_data = "..."

In [6]:
process_image(image_data)

In [25]:
res = make_api_request(
    "process", "POST", data={"imageData": image_data}, session_id=session_id
)
res

<Response [200]>

In [26]:
make_api_request("stop", "DELETE", session_id=session_id)

<Response [200]>